In [6]:
import warnings
warnings.filterwarnings("ignore")

import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import multiprocessing
from transformers import *
from datasets import load_dataset
from PIL import Image
from torchinfo import summary
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm
import time
import timm

torch.cuda.empty_cache()
device="cuda" if torch.cuda.is_available() else "cpu"

## Model Creation

In [7]:
model_name="vit_base_patch16_384.orig_in21k_ft_in1k"

model = timm.create_model(model_name, pretrained=True)

data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

## Importing Data

In [30]:
def transform(examples):
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]
  return inputs

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

model_name = "google/vit-base-patch16-224"
batch_size = 16
cpu_count=multiprocessing.cpu_count()

image_processor = ViTImageProcessor.from_pretrained(model_name)

train_ds= load_dataset('../../../Desktop_SIH/SIH/Main_Dataset/')
train_ds = train_ds["train"].train_test_split(test_size=0.25) 


#train_ds= load_dataset('chest_xray/train/',num_proc=cpu_count)
#test_ds= load_dataset('chest_xray/test/',num_proc=cpu_count)

labels = train_ds["train"].features["label"].names
#dataset_train = train_ds.with_transform(transform)
#dataset_test=test_ds.with_transform(transform)
dataset = train_ds.with_transform(transform)


train_dataset_loader = torch.utils.data.DataLoader(dataset["train"], collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
valid_dataset_loader = torch.utils.data.DataLoader(dataset["test"], collate_fn=collate_fn, batch_size=batch_size, shuffle=True)

#train_dataset_loader = torch.utils.data.DataLoader(dataset_train["train"], collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
#valid_dataset_loader = torch.utils.data.DataLoader(dataset_test["train"], collate_fn=collate_fn, batch_size=batch_size, shuffle=True)

loading configuration file preprocessor_config.json from cache at /home/moose/.cache/huggingface/hub/models--google--vit-base-patch16-224/snapshots/3f49326eb077187dfe1c2a2bb15fbd74e6ab91e3/preprocessor_config.json
size should be a dictionary on of the following set of keys: ({'width', 'height'}, {'shortest_edge'}, {'shortest_edge', 'longest_edge'}, {'longest_edge'}), got 224. Converted to {'height': 224, 'width': 224}.
Image processor ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}



Resolving data files:   0%|          | 0/14462 [00:00<?, ?it/s]